https://judge.mipt.ru/mipt_cs_on_python3/labs/lab11.html

Динамическое программирование — решение сложной задачи разбиением её на более простые подзадачи, при этом каждая подзадача решается только один раз.

Динамическое программирование очень похоже на рекурсию, при этом:

динамическое программирование сверху — это по сути рекурсия с кешированием;
динамическое программирование снизу — это переформулирование задачи в виде индуктивной последовательности подзадач, от крайнего случая к более сложным.

Задача о кузнечике — количество способов
Рассмотрим следующую задачу. На числовой прямой сидит кузнечик, который может прыгать вправо на одну или на две единицы. Первоначально кузнечик находится в точке с координатой 1. Определите количество различных маршрутов кузнечика, приводящих его в точку с координатой n.

Обозначим количество маршрутов кузнечика, ведущих в точку с координатой n, как K[n]. Прежде всего заметим, что существует ровно один маршрут из точки 1 в точку 1 — он не содержит ни одного прыжка. В точку 2 можно прыгнуть единственным способом — из точки 1.

Как вычислить K[n]? В точку кузнечик может попасть двумя способами — из точки при помощи прыжка длиной 2 и из точки прыжком длины 1. То есть число способов попасть в точку n равно сумме числа способов попасть в точку (n-1) и (n-2), что позволяет выписать рекуррентное соотношение: K[n] = K[n-1] + K[n-2].

Можно заметить, что данная задача по сути свелась к числам Фибоначчи, поэтому мы не будем выписывать её решение.

Упражнение №1
Решите задачу о количестве способов достичь точки n из точки 1, если кузнечик умеет прыгать +1, +2 и +3.

In [1]:
def count_grasshopper1(n):
    K=[1 if i==1 or i==2 else 0 for i in range(n+1)]
    for i in range(3,n+1):
        K[i]=K[i-1]+K[i-2]+K[i-3]
    return K[n]
count_grasshopper1(5)

7

Упражнение №2 \
Решите задачу о количестве способов достичь точки n из точки 1, если кузнечик умеет прыгать +1, +2 и *3. \
12345 \
1 +1\*4 \
3 +2\*1 +1\*2 \
1 +2\*2 \
1 1\*3 +1\*2 \
1 1\*3 +2\*1 \
итого 7

In [2]:
def count_grasshopper2(n):
    K=[1 if i==1 else 0 for i in range(n+1)]
    for i in range(3,n+1):
        k3= 0 if i%3==0 else K[i//3]
        K[i]=K[i-1]+K[i-2]+k3
    return K[n]
count_grasshopper2(5)

4

Задача о кузнечике со стоимостями посещения точек
Пусть кузнечик прыгает на одну или две точки вперед, а за прыжок в каждую точку необходимо заплатить определенную стоимость, различную для различных точек. Стоимость прыжка в точку i задается значением price[i] списка price. Необходимо найти минимальную стоимость маршрута кузнечика из точки 0 в точку n.

На этот раз нам необходимо модифицировать определение целевой функции. Пусть C[n] — минимальная стоимость пути из 1 в n.

Выведем рекуррентное соотношение для этой функции.Чтобы попасть в точку n мы должны попасть в неё последним прыжком из (n-1) или (n-2). Минимальные стоимости этих маршрутов будут равны С[n-1] и С[n-2] соответственно, к ним придется добавить значение price[n] за прыжок в клетку n. Но из двух клеток мы можем выбрать любую.

Нужно выбрать тот маршрут, который имеет наименьшую стоимость: C[n] = min(C[n-1], C[n-2]) + price[n]

Вычислить значение целевой функции также лучше при помощи динамического программирования, а не рекурсии.

Упражнение №3
Напишите функцию calculate_min_cost(n, price) вычисления наименьшей стоимость достижения клетки n из клетки 1

In [3]:
def calculate_min_cost(n, price):
    C=[float("inf"),price[0],price[0]+price[1]]+[0]*(n-2)
    for i in range(3,n+1):
        C[i]=min(C[i-1],C[i-2])+price[i-1]
    return C[n]
n=5
price1=[0 if i==0 else 1 for i in range(n)]#минимальное количество шагов
price2=[i+1 for i in range(n)]#минимальная сумма посещенных клеток
print(calculate_min_cost(n, price1),calculate_min_cost(n, price2),sep="\n")

2
9


Восстановление наиболее выгодной траектории
Итак, мы нашли список С, где будет записана минимальная стоимость маршрута для всех точек от 1 до n.

Но помимо нахождения наименьшей стоимости маршрута, разумеется, хотелось бы найти и сам маршрут минимальной стоимости. Такая задача называется задачей «восстановления ответа».

Для восстановления ответа будем для каждой точки запоминать номер точки prev[i], из которой кузнечик попал в точку i, если он будет передвигаться по пути минимальной стоимости. То есть prev[i] — это точка, предшествующая точке с номером i на пути минимальной стоимости (также говорят, что Prev — это массив предшественников). Как определить prev[i]? Если C[i-1] < C[i-2], то кузнечик попал в точку i из точки (i-1), поэтому prev[i] = i - 1, иначе prev[i] = i - 2.

Для восстановления пути необходимо начать с точки n и переходить от каждой точки к ее предшественнику, пока путь не дойдет до начальной точки с номером 0. Номера всех вершин будем добавлять в список path. В конце в список path добавляется начальная вершина номер 1, которая не была обработана в основном цикле, а затем весь список path разворачивается в обратном порядке (т. к. вершины добавляются в обратном порядке, от конечной к начальной).

Упражнение №4
Модифицируйте алгоритм вычисления значений целевой функции так, чтобы вычислить значения prev[i], и восстановите траекторию наименьшей стоимости из точки 1 в точку n.

In [4]:
def calculate_min_cost_traj(n, price):
    C=[float("inf"),price[0],price[0]+price[1]]+[0]*(n-2)
    prev=[-1 for i in range(n+1)]
    for i in range(3,n+1):
        C[i]=min(C[i-1],C[i-2])+price[i-1]
        prev[i]= i-1 if C[i-1]<C[i-2] else i-2
    path=[]
    i=n
    while i!=-1:
        path.append(i)
        i=prev[i]
    return path[-1::-1]
n=5
price1=[0 if i==0 else 1 for i in range(n)]#минимальное количество шагов
price2=[i+1 for i in range(n)]#минимальная сумма посещенных клеток
print(calculate_min_cost(n, price1),calculate_min_cost_traj(n,price1),
      calculate_min_cost(n, price2),calculate_min_cost_traj(n,price2),sep="\n")

2
[1, 3, 5]
9
[1, 3, 5]


Игра с ферзём
Рассмотрим игру «Ферзя в угол» для двух игроков. В левом верхнем углу доски размером N*M находится ферзь, который может двигаться только вправо-вниз. Игроки по очереди двигают ферзя, то есть за один ход игрок может переместить ферзя либо по вертикали вниз, либо по горизонтали вправо, либо во диагонали вправо-вниз. Выигрывает игрок, который поставит ферзя в правый нижний угол. Необходимо определить, какой из игроков может выиграть в этой игре независимо от ходов другого игрока (имеет выигрышную стратегию).

Будем заполнять доску знаками «+» и «-». Знак «+» будет означать, что данная клетка является выигрышной для ходящего с неё игрока (то есть если ферзь стоит в этой клетке, то игрок, который делает ход, может всегда выиграть), а знак «-» означает, что он проигрывает. Клетки последней строки, последнего столбца и диагонали, ведущей из правого нижнего угла необходимо отметить, как «+», так как если ферзь стоит в этой клетке, то ходящий игрок может выиграть одним ходом.

Но в правом нижнем углу необходимо поставить знак «-» — если ферзь стоит в углу, то тот игрок, которых должен делать ход, уже проиграл.
Но теперь в те клетки, из которых можно попасть в клетку, в которой стоит знак «-» за один ход, необходимо записать знак «+» — если ферзь стоит в этой клетке, то игрок, который делает ход, может выиграть, если передвинет ферзя в клетку, в которой стоит знак «-»:


Дальше таблица заполняется аналогично. В клетке ставиться знак «+», если есть ход, который ведет в клетку, в которой стоит знак «--». В клетке ставится знак «-», если все ходы из этой клетки ведут в клетки, в которых записан знак «+».

Продолжая таким образом, можно определить выигрывающего игрока для любой начальной клетки.


Упражнение №5
Реализовать алгоритм поиска выигрышных и проигрышных позиций в игре с ферзём на прямоугольном поле M на N, где N — высота, а M — ширина поля.
Крайний случай: \
A[N][M]=0

In [8]:
def qween_win_seach(N,M):
    A=[[1 if j==M or i==N else 0 for j in range(M+1)]for i in range(N+1)]
    A[N][M]=0
    i=N;j=M
    while i>0:
        if A[i][j]==0:
            for k in range (1,min(i,j)):
                A[i][k]=1 #строка
                A[k][j]=1 #столбец
                A[i-k][j-k]=1#диагональ
            #Дописываем строку и столбец
            if i>j:
                for k in range(j,i):
                    A[k][j]=1
            else:
                for k in range(i,j): A[i][k]=1
            i-=1
            j=M
        elif j>1:#по строке назад
            j-=1
        else: # по столбцам вверх
            i-=1
            j=M
    return A[1][1]

N,M=4,6
print("first" if qween_win_seach(N,M) else "second")

second


Упражнение №6
Реализовать алгоритм поиска выигрышных и проигрышных позиций в аналогичной игре, но ходы делает король (только вправо, вниз и по диагонали).

In [9]:
def king_win_seach(N,M):
    A=[[0 for j in range(M+1)]for i in range(N+1)]
    for i in range (N,0,-1):
        for j in range(M,0,-1):
            if A[i][j]==0:
                A[i-1][j]=A[i][j-1]=A[i-1][j-1]=1
    return A[1][1]

N,M=2,5
print("first" if king_win_seach(N,M) else "second")

first
